In [15]:
# --- SEÇÃO 0: IMPORTS ---
import pandas as pd
import numpy as np
import json
import re
import string
import joblib
import time
import os
import nltk
from unidecode import unidecode
from nltk.tokenize import WordPunctTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier

# --- Download de pacotes NLTK (necessário apenas na primeira execução) ---
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Baixando pacotes NLTK (punkt, stopwords)...")
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    print("Pacotes NLTK baixados.")

# ==============================================================================
# SEÇÃO 1: CONFIGURAÇÃO GLOBAL
# Centraliza todos os caminhos e parâmetros para fácil manutenção.
# ==============================================================================
CONFIG = {
    # Caminhos dos arquivos de entrada e saída
    "data_path": "dados/",
    "model_artifact_path": "models/decision_model.pkl",
    "jobs_list_path": "models/vagas_disponiveis.json",
    "jobs_descriptions_path": "models/descricoes_vagas.json",
    
    # Definição das colunas para feature engineering
    "text_features": [
        'cv_pt', 'cv_en', 'informacoes_profissionais.conhecimentos_tecnicos', 
        'informacoes_profissionais.certificacoes', 'perfil_vaga.principais_atividades', 
        'perfil_vaga.competencia_tecnicas_e_comportamentais', 
        'informacoes_basicas.titulo_vaga', 'perfil_vaga.demais_observacoes', 'comentario'
    ],
    "categorical_features": [
        'perfil_vaga.nivel profissional', 'perfil_vaga.nivel_academico',
        'perfil_vaga.nivel_ingles', 'formacao_e_idiomas.nivel_academico',
        'formacao_e_idiomas.nivel_ingles', 'formacao_e_idiomas.nivel_espanhol',
    ],
    "target": "aderente",
    
    # Parâmetros para o modelo e divisão dos dados
    "test_size": 0.2,
    "random_state": 42
}

# ==============================================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES
# Funções para carregar e pré-processar os dados.
# ==============================================================================

def carregar_dados_vagas_applicants(caminho_json):
    """Lê e achata os JSONs de vagas ou applicants."""
    with open(caminho_json, 'r', encoding='utf-8') as file:
        dados = json.load(file)
    linhas = []
    for chave, conteudo in dados.items():
        linha = {"ID": chave}
        for secao, valores in conteudo.items():
            if isinstance(valores, dict):
                for subchave, subvalor in valores.items():
                    linha[f"{secao}.{subchave}"] = subvalor
            else:
                linha[secao] = valores
        linhas.append(linha)
    return pd.DataFrame(linhas)

def carregar_dados_candidaturas(caminho_json):
    """Lê e processa o JSON de prospects/candidaturas."""
    with open(caminho_json, 'r', encoding='utf-8') as file:
        dados_candidatura = json.load(file)
    linhas_candidatura = []
    for codigo_vaga, conteudo in dados_candidatura.items():
        prospects = conteudo.get("prospects")
        if isinstance(prospects, list) and prospects:
            for prospect in prospects:
                linha = {
                    'id_vaga': codigo_vaga,
                    'id_candidato': prospect.get('codigo', ''),
                    'situacao_candidato': prospect.get('situacao_candidado', ''),
                    'comentario': prospect.get('comentario', '')
                }
                linhas_candidatura.append(linha)
    return pd.DataFrame(linhas_candidatura)

def limpar_e_processar_texto(texto, tokenizer, stopwords_set):
    """Executa a pipeline completa de limpeza e remoção de stopwords para um texto."""
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = re.sub(r'\\s+', ' ', texto).strip()
    texto = re.sub(f'[{re.escape(string.punctuation)}]', '', texto)
    texto = unidecode(texto)
    tokens = tokenizer.tokenize(texto)
    tokens_filtrados = [token for token in tokens if token not in stopwords_set and len(token) > 1]
    return ' '.join(tokens_filtrados)

# ==============================================================================
# SEÇÃO 3: FLUXO PRINCIPAL DE TREINAMENTO E AVALIAÇÃO
# ==============================================================================

def run_training():
    """
    Orquestra o fluxo completo de carga, processamento, treinamento, avaliação e salvamento.
    """
    print("Iniciando o processo de treinamento...")
    start_total_time = time.time()

    # --- 3.1: Carregamento e Merge dos Dados ---
    print("\n[ETAPA 1/5] Carregando e unindo os dados...")
    # df_vagas = carregar_dados_vagas_applicants(os.path.join(CONFIG['data_path'], 'vagas.json'))
    # df_applicants = carregar_dados_vagas_applicants(os.path.join(CONFIG['data_path'], 'applicants.json'))
    # df_candidaturas = carregar_dados_candidaturas(os.path.join(CONFIG['data_path'], 'prospects.json'))
    
    # df_vagas.rename(columns={'ID': 'id_vaga'}, inplace=True)
    # df_applicants.rename(columns={'ID': 'id_candidato'}, inplace=True)
    
    # df_merge = pd.merge(df_candidaturas, df_vagas, on='id_vaga', how='inner')
    # df_final = pd.merge(df_merge, df_applicants, on='id_candidato', how='inner')

    # --- 3.2: Engenharia de Features ---
    print("\n[ETAPA 2/5] Realizando engenharia de features...")
    # Criação da variável alvo 'aderente'
    dic_map_situacao = {
        'Contratado pela Decision': 1, 'Contratado como Hunting': 1, 'Aprovado': 1, 
        'Proposta Aceita': 1, 'Documentação PJ': 1, 'Documentação CLT': 1, 
        'Documentação Cooperado': 1, 'Encaminhar Proposta': 1,
        'Não Aprovado pelo Cliente': 0, 'Não Aprovado pelo Requisitante': 0, 'Recusado': 0
    }
    # df_final[CONFIG['target']] = df_final['situacao_candidato'].map(dic_map_situacao)
    # df_final.dropna(subset=[CONFIG['target']], inplace=True)
    # df_final[CONFIG['target']] = df_final[CONFIG['target']].astype(int)

    # Processamento e combinação das features de texto
    tokenizer = WordPunctTokenizer()
    stopwords_pt = set(nltk.corpus.stopwords.words('portuguese'))
    # df_final['texto_completo'] = df_final[CONFIG['text_features']].fillna('').agg(' '.join, axis=1)
    # df_final['texto_completo'] = df_final['texto_completo'].apply(lambda x: limpar_e_processar_texto(x, tokenizer, stopwords_pt))
    
    # Tratamento de valores nulos nas features categóricas
    # df_final[CONFIG['categorical_features']] = df_final[CONFIG['categorical_features']].fillna('Desconhecido')
    # print('gerando parquet do df_final')
    # df_final.to_parquet(CONFIG['data_path']+'df_final.parquet', index=False)
    # print('arquivo gerado')
    df_final = pd.read_parquet((CONFIG['data_path']+'df_final.parquet'))

    # --- 3.3: Definição do Pipeline e Divisão dos Dados ---
    print("\n[ETAPA 3/5] Definindo o pipeline e dividindo os dados...")
    X = df_final[['texto_completo'] + CONFIG['categorical_features']]
    y = df_final[CONFIG['target']]
    
    # Divisão em treino e teste para avaliação de performance
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=CONFIG['test_size'], 
        random_state=CONFIG['random_state'], 
        stratify=y  # Importante para problemas de classificação
    )
    print(f"Dados divididos em {len(X_train)} para treino e {len(X_test)} para teste.")

    # Definição do pré-processador que aplica transformações diferentes a cada tipo de coluna
    preprocessor = ColumnTransformer(
        transformers=[
            ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2)), 'texto_completo'),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), CONFIG['categorical_features'])
        ],
        remainder='drop' # Ignora colunas não especificadas
    )
    
    # Definição do pipeline final que encadeia pré-processamento e o modelo
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(
            random_state=CONFIG['random_state'], eval_metric='logloss',
            n_estimators=200, learning_rate=0.1, max_depth=5
        ))
    ])

    # --- 3.4: Treinamento para Avaliação de Performance ---
    print("\n[ETAPA 4/5] Treinando e avaliando o modelo...")
    model_pipeline.fit(X_train, y_train)
    
    y_pred = model_pipeline.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='binary')

    print("\n--- MÉTRICAS DE PERFORMANCE (NO CONJUNTO DE TESTE) ---")
    print(f"  - Acurácia: {acc:.4f}")
    print(f"  - F1-Score: {f1:.4f}")
    print("----------------------------------------------------\n")

    # --- 3.5: Treinamento Final e Salvamento dos Artefatos ---
    print("[ETAPA 5/5] Treinando modelo final com todos os dados e salvando artefatos...")
    # Retreina o pipeline com 100% dos dados para o modelo de produção
    model_pipeline.fit(X, y)
    
    # Garante que a pasta de destino exista antes de salvar
    model_dir = os.path.dirname(CONFIG['model_artifact_path'])
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # Salva o pipeline treinado
    joblib.dump(model_pipeline, CONFIG['model_artifact_path'])
    
    # Salva a lista de vagas únicas para o dropdown da aplicação
    #vagas_disponiveis = df_vagas['informacoes_basicas.titulo_vaga'].dropna().unique().tolist()
    vagas_disponiveis = df_final['informacoes_basicas.titulo_vaga'].dropna().unique().tolist()
    with open(CONFIG['jobs_list_path'], 'w', encoding='utf-8') as f:
        json.dump(vagas_disponiveis, f, ensure_ascii=False, indent=4)
        
    # Salva as descrições das vagas
    cols_descricao = ['perfil_vaga.principais_atividades', 'perfil_vaga.competencia_tecnicas_e_comportamentais', 'perfil_vaga.demais_observacoes']
    cols_descricao_e_nome = cols_descricao + ['informacoes_basicas.titulo_vaga']
    df_vagas_2 = df_final[cols_descricao_e_nome].copy()
    df_vagas_2[cols_descricao] = df_vagas_2[cols_descricao].fillna('')
    descricoes = {}
    vagas_unicas = df_vagas_2.drop_duplicates(subset=['informacoes_basicas.titulo_vaga'])
    for _, row in vagas_unicas.iterrows():
        titulo = row['informacoes_basicas.titulo_vaga']
        descricao_formatada = f"""
**Principais Atividades:**\n{row['perfil_vaga.principais_atividades']}\n\n
**Competências Técnicas e Comportamentais:**\n{row['perfil_vaga.competencia_tecnicas_e_comportamentais']}\n\n
**Observações Adicionais:**\n{row['perfil_vaga.demais_observacoes']}
"""
        descricoes[titulo] = descricao_formatada.strip()

    with open(CONFIG['jobs_descriptions_path'], 'w', encoding='utf-8') as f:
        json.dump(descricoes, f, ensure_ascii=False, indent=4)
        
    end_total_time = time.time()
    print(f"\nProcesso concluído com sucesso em {end_total_time - start_total_time:.2f} segundos.")
    print(f"Modelo e artefatos salvos na pasta: '{model_dir}'")

# ==============================================================================
# PONTO DE ENTRADA DO SCRIPT
# ==============================================================================
if __name__ == "__main__":
    run_training()

Iniciando o processo de treinamento...

[ETAPA 1/5] Carregando e unindo os dados...

[ETAPA 2/5] Realizando engenharia de features...

[ETAPA 3/5] Definindo o pipeline e dividindo os dados...
Dados divididos em 4991 para treino e 1248 para teste.

[ETAPA 4/5] Treinando e avaliando o modelo...

--- MÉTRICAS DE PERFORMANCE (NO CONJUNTO DE TESTE) ---
  - Acurácia: 0.8221
  - F1-Score: 0.7910
----------------------------------------------------

[ETAPA 5/5] Treinando modelo final com todos os dados e salvando artefatos...

Processo concluído com sucesso em 65.96 segundos.
Modelo e artefatos salvos na pasta: 'models'


In [3]:
df_teste = pd.read_parquet(CONFIG['data_path']+'df_final.parquet')

In [6]:
print('parquet',df_teste.shape)
#print('df_final', df_final.shape)
df_teste.head()

parquet (6239, 107)


,id_vaga,id_candidato,situacao_candidato,comentario,informacoes_basicas.data_requicisao,informacoes_basicas.limite_esperado_para_contratacao,informacoes_basicas.titulo_vaga,informacoes_basicas.vaga_sap,informacoes_basicas.cliente,informacoes_basicas.solicitante_cliente,...,cargo_atual.cargo_atual,cargo_atual.projeto_atual,cargo_atual.cliente,cargo_atual.unidade,cargo_atual.data_admissao,cargo_atual.data_ultima_promocao,cargo_atual.nome_superior_imediato,cargo_atual.email_superior_imediato,aderente,texto_completo
0,4531,25364,Contratado pela Decision,Data de Inicio: 12/04/2021,10-03-2021,00-00-0000,2021-2607395-PeopleSoft Application Engine-Dom...,Não,Gonzalez and Sons,Valentim Duarte,...,None,None,None,None,None,None,None,None,1,area atuacao lider consultoria gerenciamento p...
1,4533,26338,Contratado pela Decision,,11-03-2021,01-01-1970,2021-2605708-Microfocus Application Life Cycle...,Não,Barnes-Woods,Maysa Andrade,...,None,None,None,None,None,None,None,None,1,solteiro brasileiro 21061987 habilitacao categ...
2,4534,26361,Documentação PJ,Aguardando confirmação de inicio _,11-03-2021,01-01-1970,2021-2605711-Microfocus QTP - UFT Automation T...,Não,Barnes-Woods,Maysa Andrade,...,None,None,None,None,None,None,None,None,1,alta mobilidade mudancas viagens objetivo atua...
3,4534,26003,Não Aprovado pelo Cliente,"""Conversando com a candidata, foi exposto que ...",11-03-2021,01-01-1970,2021-2605711-Microfocus QTP - UFT Automation T...,Não,Barnes-Woods,Maysa Andrade,...,None,None,None,None,None,None,None,None,0,solteira 40 anos brasileira itaquaquecetuba ob...
4,4534,8838,Não Aprovado pelo Cliente,Candidato reprovado pelo cliente . Candidato a...,11-03-2021,01-01-1970,2021-2605711-Microfocus QTP - UFT Automation T...,Não,Barnes-Woods,Maysa Andrade,...,None,None,None,None,None,None,None,None,0,dados pessoais nascimento 971977 reside baruer...


In [8]:
df_teste[['perfil_vaga.principais_atividades', 'perfil_vaga.competencia_tecnicas_e_comportamentais', 'perfil_vaga.demais_observacoes']]

,perfil_vaga.principais_atividades,perfil_vaga.competencia_tecnicas_e_comportamentais,perfil_vaga.demais_observacoes
0,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,"Remoto DEPOIS PRESENCIAL, TEMPO INDETERMINADO"
1,Arquiteto\n\nFoco na área e automação.\n\nRequ...,Arquiteto\n\nFoco na área e automação.\n\nRequ...,Atuação somente em horário comercial. Tempo in...
2,Automação de teste (conhecimento do código)\n\...,Automação de teste (conhecimento do código)\n\...,
3,Automação de teste (conhecimento do código)\n\...,Automação de teste (conhecimento do código)\n\...,
4,Automação de teste (conhecimento do código)\n\...,Automação de teste (conhecimento do código)\n\...,
...,...,...,...
6234,Great experience with Debug projects\nFunction...,Configuration localization Brazil – TAXBRA.,
6235,ID: 4399\nINFORMAÇÕES SOBRE A HABILIDADE\nPerf...,"Qualificações Requeridas:\n""Obrigatório:\n• Co...",Encaminhar 3 CVs em inglês.
6236,Vaga: SAP CO Consultant_L4 SAP CO Consultant_L...,Descrição da vaga: Application Consultant / Pa...,Tipo de Contratação: PJ Limite de CVS: 5 Dispo...
6237,Vaga: Tech Lead Cyber 14688495\nPeríodo de Alo...,Disponibilidade para Viagens: Não se aplica\nD...,Hunting Tipo de Contratação: CLT Cliente SLA: ...
